In [2]:
%load_ext autoreload
%autoreload 2 

In [11]:
import torch
from gptmodel import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [ ]:
import tiktoken 
from generate import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("output text: \n", token_ids_to_text(token_ids, tokenizer))

Linear(in_features=768, out_features=768, bias=False)
torch.Size([1, 4, 768])
keys shape: tensor([[[ 0.0932,  0.6565, -0.2126,  ...,  0.4595, -0.5147,  0.7862],
         [ 0.6140, -0.1549,  0.1592,  ..., -0.9458,  0.1576,  0.8178],
         [ 0.4204,  0.0710, -0.2487,  ..., -0.8524, -0.0540,  0.2318],
         [-0.0716, -1.0925, -0.2946,  ...,  1.5382,  0.2825, -0.4400]]])
keys after view shape: torch.Size([1, 4, 12, 64])
keys after transposed shape: torch.Size([1, 12, 4, 64])
Linear(in_features=768, out_features=768, bias=False)
torch.Size([1, 4, 768])
keys shape: tensor([[[-0.2203,  0.3531, -0.9539,  ..., -1.3319, -0.2020, -0.0356],
         [ 0.3896, -0.2308, -0.2119,  ...,  0.4203,  0.1190, -0.5601],
         [ 0.5144,  0.4400, -0.3474,  ..., -0.6985,  1.1897,  1.1756],
         [ 0.8182,  1.0879,  0.4619,  ...,  0.1327,  0.8847, -0.4570]]])
keys after view shape: torch.Size([1, 4, 12, 64])
keys after transposed shape: torch.Size([1, 12, 4, 64])
Linear(in_features=768, out_features